# 라이브러리 불러오기

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

# 사용 툴 : albumentation, tta
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda')

#### 기본값(시드 설정)

In [ ]:
def seed_all(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_all(42)

In [ ]:
os.chdir("/content/drive/MyDrive/DL_Project")

In [ ]:
train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))

In [ ]:
len(train_png), len(test_png)

(4277, 2154)

In [ ]:
train_y = pd.read_csv("/content/drive/MyDrive/DL_Project/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

# image processing

- grayscale을 써 봤으나 저희 팀의 경우엔 극적인 변화가 없었습니다. 
- 사이즈에 맞는 모델인 b4, b1을 주로 사용했습니다.

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (384, 384))
    return img

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)] # train_imgs 다운로드
test_imgs = [img_load(n) for n in tqdm(test_png)] # test_imgs 다운로드


100%|██████████| 4277/4277 [03:33<00:00, 20.07it/s]

100%|██████████| 2154/2154 [13:11<00:00,  2.72it/s]


## 우리가 처음에 시도했던 부분
- 1. good, bad 별로 사진을 나눠서 good 개수와 bad 개수를 맞춰 데이터 증강을 시도(15 * 2개)

- 2. label 별로 사진들을 나눠서 good을 제외힌 모든 label 사진들을 good 개수에 맞춰 데이터 증강을 시도 (88개)

- 3. 데이터 증강을 임의의 다섯 개 label만 따로 골라서 하기 (21~28개)

다른 분들이 했을 땐 다를 수 있었겠지만 저희 조는 전부 이전에 비해 결과가 안 좋게 나오거나 모델 자체가 돌아가지 않았습니다.

## 해결책
- 기본으로 돌아가서 데이터 자체의 조작을 주지 않고 모델 안에서만 데이터 조작을 해주자!

# Custom Dataset
> augmentation : augmentation 함수를 따로 만들지 않고 안에 넣어줬습니다.

> test 데이터셋을 보고 데이터에 큰 영향을 준 수평, 수직 뒤집, 랜덤 회전, affine 변환만 썼습니다.

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
            train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees = (-90, 90)),
                transforms.RandomCrop(340)        
            ])
            img = train_transform(img)
        if self.mode == 'test':
            test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
            img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
            self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
            self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

## score, data loader
이 파트는 데이콘의 기본 모델을 베이스로 구성했습니다.

In [ ]:
# 여기는 score를 계산하는 것으로 보입니다.
def score_function(real, pred):
  score = f1_score(real, pred, average="macro")
  return score

In [ ]:
# dataset : 전체 dataset 구성 dataloader : mini batch 만드는 역할할
batch_size = 32 # batch_size : 사진들을 몇 개 묶음으로 할 거냐
epochs = 60 # 학습 시도 횟수

# 데이터 셋과 데이터 로더 부분

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train') # train 데이터셋 학습 모델델
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test') # test 데이터셋 학습 모델델
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# train 모델

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score # 모델 스코어

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs): # 학습 기본 설정 setting
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()   
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s') # 시간 확인 안내내
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    if train_f1 > 0.99:
      model_path = "/content/drive/MyDrive/open/model"
      torch.save(model.state_dict(), f"{train_f1}.pt")

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b4_ra2_320-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_ra2_320-7eb33cd5.pth


epoch : 1/60    time : 183s/10781s
TRAIN    loss : 0.96620    f1 : 0.25883
epoch : 2/60    time : 156s/9076s
TRAIN    loss : 0.43438    f1 : 0.47311
epoch : 3/60    time : 160s/9094s
TRAIN    loss : 0.32788    f1 : 0.59285
epoch : 4/60    time : 163s/9140s
TRAIN    loss : 0.26769    f1 : 0.66706
epoch : 5/60    time : 157s/8655s
TRAIN    loss : 0.20732    f1 : 0.71286
epoch : 6/60    time : 158s/8520s
TRAIN    loss : 0.19012    f1 : 0.75650
epoch : 7/60    time : 158s/8368s
TRAIN    loss : 0.15558    f1 : 0.81929
epoch : 8/60    time : 157s/8189s
TRAIN    loss : 0.15730    f1 : 0.80172
epoch : 9/60    time : 161s/8223s
TRAIN    loss : 0.13425    f1 : 0.83391
epoch : 10/60    time : 160s/7975s
TRAIN    loss : 0.14152    f1 : 0.84640
epoch : 11/60    time : 157s/7717s
TRAIN    loss : 0.12706    f1 : 0.85757
epoch : 12/60    time : 157s/7529s
TRAIN    loss : 0.10415    f1 : 0.87231
epoch : 13/60    time : 156s/7345s
TRAIN    loss : 0.08953    f1 : 0.90173
epoch : 14/60    time : 157s/7220

### 스코어 결과
- 1. baseline model : 69.0 (모델 : b4, size = 384)
- 2. augmentation model : 75.5 (모델 : b4, size = 256, batch-size : 64)
- 3. augmentation model(b1) : 73.3 (모델 : b1, size = 256, batch-size : 32)

> final model : public : 79.0, private : 80.3
> (모델 : b4, size = 384, epoch : 60, batch-size : 32) 

# 모델 추론


In [ ]:
model.eval()
f_pred = [] 

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

label_decoder = {val:key for key, val in label_unique.items()} 

f_result = [label_decoder[result] for result in f_pred]        

# 결과창

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/DL_Project/sample_submission.csv") # 데이터 제출출

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [ ]:
submission[['index','label']].to_csv("/content/drive/MyDrive/DL_Project/sample_submission_1.csv", index=False)

## 앙상블 기법
- 여러분이 아시는 그 앙상블로, 파라미터나 epoch, batch, resize 등 다양한 기법으로 만들어진 모델 중 
- 괜찮은 것들을 찾아 앙상블하는 것입니다.
- 최종적으로 시간 부족으로 실행하지 못하였습니다. 이 부분은 초반에 헤맨 부분이 많아서 개인적으로 아쉽네요.

## 아쉬운 점
- train 정확도가 99%를 찍지 못해서 원하는 모델을 추출하지 못한게 아쉬움
- 모델 앙상블 기법을 제대로 시도하지 못한 것도 아쉬움
- 512 size와 b6~b7 모델을 시험하지 못했음(colab 이슈)

# train 코드 (reference)
- 데이콘의 다른 예시들이 코드를 실험하는 데 큰 도움이 되었습니다.

In [ ]:
# 시도 횟수
n_epochs = 100
valid_loss_min = np.inf # 100보다 작을 시 멈춤

# keep track of training and validation loss
train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)

train_F1 = torch.zeros(n_epochs)
valid_F1 = torch.zeros(n_epochs)
model.to(device)

for e in range(0, n_epochs):

   
    ###################
    # 모델 학습       #
    ###################
    model.train()
    for data, labels in tqdm(train_dataloader):
        # move tensors to GPU if CUDA is available
        data, labels = data.to(device), labels.to(device)
        # 순전파 :compute predicted outputs by passing inputs to the model
        logits = model(data)
        # 배치의 손실율 계산
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        # 역전파 : compute gradient of the loss with respect to model parameters
        loss.backward()
        # 싱글 옵티미제이션(adam)
        optimizer.step()
        # 학습 손실율 업데이트
        train_loss[e] += loss.item()
        # 학습 스코어 계산산
        logits=logits.argmax(1).detach().cpu().numpy().tolist()
        labels=labels.detach().cpu().numpy().tolist()

        train_F1[e] += score_function(labels,logits)

    train_loss[e] /= len(train_dataloader)
    train_F1[e] /= len(train_dataloader)
        
        
    ######################    
    # 검증 모델          #
    ######################
    with torch.no_grad(): 
        model.eval()
        for data, labels in tqdm(valid_dataloader):
            # move tensors to GPU if CUDA is available
            data, labels = data.to(device), labels.to(device)
            # 순전파: compute predicted outputs by passing inputs to the model
            logits = model(data)
            # 배치 손실율 계산
            loss = criterion(logits, labels)
            # 평균 검증 손실율 
            valid_loss[e] += loss.item()
            # update training score
            logits=logits.argmax(1).detach().cpu().numpy().tolist()
            labels=labels.detach().cpu().numpy().tolist()
            valid_F1[e] += score_function(labels,logits)
            
    
    # 평균 손실율 계산산
    valid_loss[e] /= len(valid_dataloader)
    valid_F1[e] /= len(valid_dataloader)
    
    scheduler.step(valid_loss[e])    
    # 학습/검증 결과 산출 표시(loss)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss[e], valid_loss[e]))
    
    # 학습/검증 결과 산출 표시(정확도도)
    print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
        e, train_F1[e], valid_F1[e]))
    
    # 검증 손실율이 줄어들었을 때 모델 저장장
    if valid_loss[e] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss[e]))
        torch.save(model.state_dict(), 'swin_tiny_patch4_window7_224.pt')
        valid_loss_min = valid_loss[e]

## 데이터 증강 alpha 테스트(reference)

- albumentation과 transform 고려
- 최종적으로는 transform을 사용(별 이유는 없음)

In [ ]:
# 하는 방법
import albumentations
import albumentations.pytorch

aug = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.HorizontalFlip(), #수평 뒤집기기 
      albumentations.VerticalFlip(), #수직 뒤집기
      albumentations.OneOf([
                          albumentations.Rotate(), # 돌리기기
                          albumentations.ShiftScaleRotate()
 
      ], p=1),
      albumentations.augmentations.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), #정규화화
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])
aug2 = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.Rotate(), # 돌리기기
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

## 후일담

- 나중엔 코드를 정렬해서 전역 파라미터를 한 자리에서 설정할 수 있게 코드를 짜 보고 싶음 (밑 코드 참조)



In [ ]:
config = { # 전역 파라미터 변수 설정정
    # Model parameters
    'model': 'efficientnet_b0',
    'batch_size': 32,
    'pretrain': True,
    
    # Optimizer parameters
    'optimizer': 'AdamW',
    'lr': 2e-4,
    'lr_t': 15,
    'lr_scheduler': 'CosineAnnealingWarmUpRestarts',
    'gamma': 0.524,
    'loss_function': 'CE_with_Lb',
    'patience': 10,
    'weight_decay': 0.002157,
    'label_smoothing': 0.8283,
    
    # Training parameters
    'epochs': 200,
    'n_fold': 5,
    'num_workers': 16,
    'text': "A",
    'device': '0,1,2,3'
    }

In [ ]:
def get_args_parser(): # 기본 인자값 부여로 보임임
    parser = argparse.ArgumentParser('PyTorch Inference', add_help=False)

    # Inference parameters
    parser.add_argument('--model_save_name', nargs='+', default='load_model', type=str)
    parser.add_argument('--model', default='efficientnet_b7', type=str)
    parser.add_argument('--batch_size', default=32, type=int)
    parser.add_argument('--pretrain', default=True, type=str2bool)
    parser.add_argument('--n_fold', default=5, type=int)
    parser.add_argument('--num_workers', default=16, type=int)
    parser.add_argument('--device', default='0,1,2,3', type=str)
    parser.add_argument('--tta', default=True, type=str2bool)
    parser.add_argument('--save_name', default='default', type=str)
    ..........